# Model devepment

In this document we develop and compare different models for our model devepment. We have the following sections:

1. Model creation
2. Model evaluation
3. Model implementation on test data

Note that for model creation instead of running the code each time one can load the best model.



### Import libraries

In [2]:
from preprocessing import preprocessor
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer, roc_auc_score, recall_score, roc_curve
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats import uniform, randint
import joblib


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


### Data collection

In [ ]:
# Load the dataset
total_df = pd.read_csv('../Data/Base.csv')

# Define features (X) and target (y)
X = total_df.drop(columns=['fraud_bool'])
y = total_df['fraud_bool']

# Split the data into training and test sets using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Data has been loaded")

# Apply the preprocessor to the training and test datasets
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


Data has been loaded


## 1. Model Creation

### 1a. Hyperparameter specification

We will use the following hyperparameters for our models. They are based on common hyperparameters used for the respective models with ranges that are commonly used.

In [21]:
# Pipeline with placeholder classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])

# Define models and hyperparameters
models = {
    'Logistic Regression (lbfgs)': (
        LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
        {
            'classifier__C': uniform(0.01, 10),  # Regularization strength
            'classifier__solver': ['lbfgs'],  # Only lbfgs solver
            'classifier__penalty': ['l2', 'none'],  # Supported penalties
        }
    ),
    # 'Logistic Regression (liblinear)': (
    #     LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
    #     {
    #         'classifier__C': uniform(0.01, 10),  # Regularization strength
    #         'classifier__solver': ['liblinear'],  # Only liblinear solver
    #         'classifier__penalty': ['l1', 'l2'],  # Supported penalties
    #     }
    # ),
    # 'Logistic Regression (saga)': (
    #     LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
    #     {
    #         'classifier__C': uniform(0.01, 10),  # Regularization strength
    #         'classifier__solver': ['saga'],  # Only saga solver
    #         'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],  # All supported penalties
    #         'classifier__l1_ratio': uniform(0, 1),  # ElasticNet mixing parameter (only with saga and elasticnet)
    #     }
    # ),
    # 'Logistic Regression (default)': (
    #     LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
    #     {
    #         'classifier__C': uniform(0.01, 10),  # Regularization strength
    #     }
    # ),
    # 'Random Forest': (
    #     RandomForestClassifier(class_weight='balanced', random_state=42),
    #     {
    #         'classifier__n_estimators': randint(100, 300),  # Number of trees
    #         'classifier__max_depth': randint(5, 20),  # Max tree depth
    #         'classifier__min_samples_split': randint(2, 10),  # Min samples to split
    #         'classifier__min_samples_leaf': randint(1, 5),  # Min samples per leaf
    #         'classifier__class_weight': ['balanced', {0: 1, 1: 10}],  # Explicit weights
    #     }
    # ),
    # 'Gradient Boosting': (
    #     GradientBoostingClassifier(random_state=42),
    #     {
    #         'classifier__n_estimators': randint(50, 300),  # Number of boosting stages
    #         'classifier__learning_rate': uniform(0.01, 0.2),  # Shrinkage rate
    #         'classifier__max_depth': randint(3, 15),  # Max tree depth
    #         'classifier__min_samples_split': randint(2, 10),  # Min samples to split
    #         'classifier__min_samples_leaf': randint(1, 5),  # Min samples per leaf
    #     }
    # ),
    'XGBoost': (
        XGBClassifier(random_state=42, eval_metric='logloss', device='cuda', tree_method='hist'),
        {
            'classifier__n_estimators': randint(50, 300),  # Number of boosting stages
            'classifier__learning_rate': uniform(0.01, 0.2),  # Learning rate
            'classifier__max_depth': randint(3, 15),  # Maximum tree depth
            'classifier__gamma': uniform(0, 0.5),  # Min split loss
            'classifier__scale_pos_weight': [1, 5, 10, 20],  # Reweighting for imbalanced data
        }
    ),
    'LightGBM': (
        LGBMClassifier(random_state=42),
        {
            'classifier__n_estimators': randint(50, 300),  # Number of boosting stages
            'classifier__learning_rate': uniform(0.01, 0.2),  # Learning rate
            'classifier__max_depth': randint(3, 15),  # Maximum tree depth
            'classifier__num_leaves': randint(20, 50),  # Number of leaves
            # 'classifier__class_weight': ['balanced', {0: 1, 1: 10}],  # Explicit weights
        }
    ),
    # 'Naive Bayes': (
    #     GaussianNB(),
    #     {}  # No hyperparameters for Naive Bayes
    # ),
}

### 1b. Exploratory Hyperparameter Search 

To find the best hyperparameters for each model we will use RandomizedSearchCV because it is more efficient than GridSearchCV for the exploration of the hyperparameter space.

Note: Running the following code is not recommended. It is just for reference. Running it will train the models, and we have provided our training mnodels in the repository.

In [22]:
import time
import joblib
import os
from datetime import datetime
from tqdm import tqdm # Progress bar
from glob import glob # File search
from sklearn.metrics import roc_curve
import numpy as np

# Custom recall scorer at a target FPR threshold
def recall_at_fpr(y_true, y_scores, target_fpr=0.05):
    # Ensure y_true contains at least one positive and one negative sample
    if len(np.unique(y_true)) < 2:
        return np.nan  # Return NaN if there's no variability in the target

    # Ensure y_scores is valid
    if np.any(np.isnan(y_scores)) or np.any(np.isinf(y_scores)):
        return np.nan  # Return NaN for invalid scores

    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)

    # If no threshold meets the target FPR, return 0.0
    if len(np.where(fpr <= target_fpr)[0]) == 0:
        return 0.0

    # Get the highest threshold index where FPR is <= target FPR
    threshold_idx = np.where(fpr <= target_fpr)[0][-1]
    return tpr[threshold_idx]

# Wrapper function to pass the target FPR
def recall_at_fpr_scorer(y_true, y_scores):
    return recall_at_fpr(y_true, y_scores, target_fpr=0.05)

# Make a scorer for recall @ 5% FPR
from sklearn.metrics import make_scorer
recall_5_fpr_scorer = make_scorer(recall_at_fpr_scorer, greater_is_better=True, needs_proba=True)

# Create directory for saving models if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

# Stratified K-Fold Cross-Validation
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Number of random searches per model
n_iter_per_model = 5
best_models = []

# Dictionary to store all RandomizedSearchCV objects
search_results = {}

# Iterate through each model
for name, (model, params) in models.items():
    print(f"\n--- Starting RandomizedSearchCV for {name} ---\n")
    
    # Check if the model has already been trained and skip if it exists
    saved_files = glob(f"saved_models/{name}_*.joblib")
    if saved_files:
        print(f"Model {name} already trained. Skipping...")
        continue

    # Create pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    # Create RandomizedSearchCV
    search = RandomizedSearchCV(
        pipeline, 
        param_distributions=params,
        n_iter=n_iter_per_model,
        cv=stratified_cv,
        n_jobs=1,
        random_state=42,
        scoring=recall_5_fpr_scorer,
        verbose=3
    )

    # Start timing
    start_time = time.time()

    # Fit the model
    try:
        print(f"Fitting the model {name} with {n_iter_per_model} iterations and {stratified_cv.get_n_splits()} cross-validation splits...")
        search.fit(X_train, y_train)
    except Exception as e:
        print(f"An error occurred while fitting {name}: {e}")
        continue

    # End timing
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time
    elapsed_time_str = f"{elapsed_time:.2f}s"
    print(f"\n--- Finished RandomizedSearchCV for {name} in {elapsed_time_str} ---\n")

    # Store the search object in the dictionary
    search_results[name] = search

    # Get current date and time for naming
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Create a unique filename with version number, model name, date, and training time
    filename = f"saved_models/{name}_{current_datetime}_{elapsed_time_str}.joblib"
    
    # Save each RandomizedSearchCV object immediately after training
    print(f"Saving RandomizedSearchCV results for {name} as {filename}...\n")
    joblib.dump(search, filename)
    print(f"RandomizedSearchCV for {name} saved successfully.\n")

print("\n--- All Models Processed ---\n")


/home/gamacore/FINAL/Bank-Fraud-Detection/bank/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(



--- Starting RandomizedSearchCV for Logistic Regression (lbfgs) ---

Model Logistic Regression (lbfgs) already trained. Skipping...

--- Starting RandomizedSearchCV for XGBoost ---

Model XGBoost already trained. Skipping...

--- Starting RandomizedSearchCV for LightGBM ---

Fitting the model LightGBM with 5 iterations and 5 cross-validation splits...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[LightGBM] [Info] Number of positive: 7059, number of negative: 632941
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.092195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2318
[LightGBM] [Info] Number of data points in the train set: 640000, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011030 -> initscore=-4.496074
[LightGBM] [Info] Start training from score -4.496074
[CV 1/5] END classifier__l

### 1c. Search saving and loading

Save search:

In [ ]:
import os
import joblib
# Define the directory containing the saved joblib files
saved_models_dir = 'saved_models'

# Initialize the final search_results dictionary to store all models
search_results = {}

# Iterate over each joblib file in the directory
for filename in os.listdir(saved_models_dir):
    if filename.endswith(".joblib"):
        # Construct the full path to the file
        filepath = os.path.join(saved_models_dir, filename)

        # Load the model information from the file
        model_info = joblib.load(filepath)

        # Use the filename (without extension) as the key in the search_results dictionary
        model_key = filename.replace('.joblib', '')
        
        # Add the loaded model information to the search_results dictionary
        search_results[model_key] = model_info

# Save the combined search_results dictionary to a new joblib file
combined_filename = 'aaasearch_results.joblib'
joblib.dump(search_results, combined_filename)
print(f"Combined search results saved successfully as {combined_filename}")


/home/gamacore/FINAL/Bank-Fraud-Detection/bank/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: Can't get attribute 'recall_at_fpr_scorer' on <module '__main__'>

In [ ]:
import os
import joblib
# Define the directory containing the saved joblib files
saved_models_dir = 'saved_models'

# Initialize the final search_results dictionary to store all models
search_results = {}

# Iterate over each joblib file in the directory
for filename in os.listdir(saved_models_dir):
    if filename.endswith(".joblib"):
        # Construct the full path to the file
        filepath = os.path.join(saved_models_dir, filename)

        # Load the model information from the file
        model_info = joblib.load(filepath)

        # Use the filename (without extension) as the key in the search_results dictionary
        model_key = filename.replace('.joblib', '')
        
        # Add the loaded model information to the search_results dictionary
        search_results[model_key] = model_info

# Save the combined search_results dictionary to a new joblib file
combined_filename = 'aaasearch_results.joblib'
joblib.dump(search_results, combined_filename)
print(f"Combined search results saved successfully as {combined_filename}")


Load search:

In [28]:
search_results = joblib.load(f"search_results.joblib")

## 2. Model evaluation

### 2a. Random search evaluation

Due to the biased data set we use the auc roc score to evaluate different models. We start by printing the best model of each model class for our search. 

In [29]:
# Placeholder for storing extended metrics
results_summary = []

# Collect extended metrics for each trained model
for model_name, search in search_results.items():
    # Retrieve the best estimator from the search object
    best_model = search.best_estimator_

    # Predict probabilities on the test data (assuming test data exists as X_test, y_test)
    y_scores = best_model.predict_proba(X_test)[:, 1]  # Probability for positive class
    auc_score = roc_auc_score(y_test, y_scores)

    # Calculate TPR at 5% FPR
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    target_fpr = 0.05

    # Find the TPR and threshold at target FPR
    if len(np.where(fpr <= target_fpr)[0]) > 0:
        threshold_idx = np.where(fpr <= target_fpr)[0][-1]
        tpr_at_fpr = tpr[threshold_idx]
        threshold_at_fpr = thresholds[threshold_idx]
    else:
        tpr_at_fpr = 0.0
        threshold_at_fpr = None

    # Append model results
    results_summary.append({
        'Model': model_name,
        'Best Score (AUC)': f"{auc_score:.4f}",
        'TPR @ 5% FPR': f"{tpr_at_fpr * 100:.2f}%",
        'Threshold @ 5% FPR': f"{threshold_at_fpr:.2f}" if threshold_at_fpr is not None else "N/A",
        'Best Parameters': search.best_params_
    })

# Convert to a DataFrame and sort by AUC score
results_df = pd.DataFrame(results_summary).sort_values(by='Best Score (AUC)', ascending=False)

# Display the DataFrame in Jupyter
from IPython.display import display

display(results_df)

,Model,Best Score (AUC),TPR @ 5% FPR,Threshold @ 5% FPR,Best Parameters
6,LightGBM_2024-12-09_02-24-26_150.89s,0.8878,53.54%,0.04,{'classifier__learning_rate': 0.03857336358438...
4,Gradient Boosting_2024-12-09_01-48-51_20030.52s,0.8874,52.99%,0.04,{'classifier__learning_rate': 0.04120372808848...
3,XGBoost_2024-12-09_02-19-15_209.47s,0.8809,51.45%,0.22,"{'classifier__gamma': 0.2984250789732435, 'cla..."
5,Random Forest_2024-12-08_20-15-00_3690.86s,0.8759,51.81%,0.16,"{'classifier__class_weight': {0: 1, 1: 10}, 'c..."
0,Logistic Regression (lbfgs)_v1_2024-12-08_19-0...,0.8740,48.73%,0.81,"{'classifier__C': 4.468327528535911, 'classifi..."
2,Logistic Regression (default)_2024-12-08_19-11...,0.8740,48.78%,0.81,{'classifier__C': 9.51714306409916}
1,Naive Bayes_2024-12-09_01-53-13_19.49s,0.8416,42.79%,0.84,{}


### 2b. Best model evaluation

We begin the evaluation of the best model by extracting it from search_results.

In [30]:
# Initialize variables to track the best model
best_model_name = None
best_model_score = -float('inf')
best_model_params = None
best_model_object = None
best_classifier = None

# Iterate through the search results to find the best model
for model_name, search in search_results.items():
    if search.best_score_ > best_model_score:
        best_model_name = model_name
        best_model_score = search.best_score_
        best_model_params = search.best_params_
        best_model_object = search.best_estimator_

        # Extract the classifier from the pipeline
        best_classifier = best_model_object.named_steps['classifier']

# Print the best model details
print(f"Best Model Name: {best_model_name}")
print(f"Best Model Score (AUC): {best_model_score:.4f}")
print(f"Best Model Parameters: {best_model_params}")

# Print the best classifier object
print(f"Best Classifier Object: {best_classifier}")


Best Model Name: Gradient Boosting_2024-12-09_01-48-51_20030.52s
Best Model Score (AUC): 0.5196
Best Model Parameters: {'classifier__learning_rate': 0.0412037280884873, 'classifier__max_depth': 5, 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 4, 'classifier__n_estimators': 252}
Best Classifier Object: GradientBoostingClassifier(learning_rate=0.0412037280884873, max_depth=5,
                           min_samples_leaf=3, min_samples_split=4,
                           n_estimators=252, random_state=42)


### Continued Model Development

Given that LightGBM and XGBoost are the top performing models, we will continue to develop these models further. We will start by tuning the hyperparameters of the models using Optuna, which is a library that applies Bayesian optimization to hyperparameter tuning. They adaptively explore the search space, focusing on more promising hyperparameter sets, which theoretically makes them more effective than RandomizedSearchCV.

In [62]:
models = {
    # "LightGBM": {
    #     "model": "lgb.LGBMClassifier",
    #     "param_distributions": {
    #         "classifier__num_leaves": [31, 63, 127],
    #         "classifier__max_depth": [-1, 5, 10, 20],
    #         "classifier__learning_rate": uniform(loc=0.01, scale=0.09),
    #         "classifier__n_estimators": randint(low=100, high=1000),
    #         "classifier__min_child_weight": [1, 10, 100],
    #         "classifier__subsample": uniform(loc=0.5, scale=0.5),  # Fix here
    #         "classifier__colsample_bytree": uniform(loc=0.5, scale=0.5),
    #         "classifier__scale_pos_weight": [10, 50, 100, 500],
    #         "classifier__boosting_type": ["gbdt", "dart"],
    #         "classifier__objective": ["binary"],
    #         "classifier__metric": ["auc"]
    #     }
    # },
    "XGBoost": {
        "model": XGBClassifier(random_state=42, eval_metric="aucpr", device="cuda", tree_method="hist"),
        "param_distributions": {
            "classifier__max_depth": randint(low=3, high=10),
            "classifier__learning_rate": uniform(loc=0.01, scale=0.09),
            "classifier__n_estimators": randint(low=100, high=1000),
            "classifier__subsample": uniform(loc=0.5, scale=0.5),  # Fix here
            "classifier__colsample_bytree": uniform(loc=0.5, scale=0.5),
            "classifier__min_child_weight": [1, 5, 10, 50],
            "classifier__scale_pos_weight": [10, 50, 100, 500],
            "classifier__gamma": uniform(loc=0, scale=5),
            "classifier__reg_alpha": uniform(loc=0, scale=1),
            "classifier__reg_lambda": uniform(loc=0, scale=1),
            "classifier__objective": ["binary:logistic"],
            "classifier__eval_metric": ["aucpr"],
        }
    }
}


In [63]:
import json
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
import joblib
import os
from datetime import datetime
import time
from scipy.stats import randint, uniform
from scipy.stats._distn_infrastructure import rv_frozen  # For checking scipy distributions
from scipy.stats import rv_continuous, rv_discrete

# Ensure output directory exists
os.makedirs('saved_models', exist_ok=True)

# Perform stratified cross-validation
scores = []
for train_idx, val_idx in stratified_cv.split(X_train, y_train):
    # Use iloc for positional indexing
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    pipeline.fit(X_train_fold, y_train_fold)
    y_val_scores = pipeline.predict_proba(X_val_fold)[:, 1]
    score = recall_at_fpr(y_val_fold, y_val_scores, target_fpr=0.05)
    scores.append(score)


# Optuna optimization for each model
best_models = {}
search_results = {}

def save_study_progress(study, filename):
    """Save the study progress to a file."""
    joblib.dump(study, filename)


def objective(trial, model_name, model, param_distributions, X_train, y_train):
    # Define hyperparameters for the trial
    params = {}
    for key, value in param_distributions.items():
        if isinstance(value, list):  # Handle list values
            params[key.split("__")[1]] = trial.suggest_categorical(key, value)
        elif isinstance(value, rv_frozen):  # Handle scipy.stats frozen distributions
            if isinstance(value.dist, rv_discrete):  # Check if it is a discrete distribution
                low = value.kwds.get("low", None)
                high = value.kwds.get("high", None)
                if low is None or high is None:
                    raise ValueError(f"Invalid randint distribution for {key}. 'low' or 'high' is missing.")
                params[key.split("__")[1]] = trial.suggest_int(key, int(low), int(high - 1))
            elif isinstance(value.dist, rv_continuous):  # Check if it is a continuous distribution
                loc = value.kwds.get("loc", None)
                scale = value.kwds.get("scale", None)
                if loc is None or scale is None:
                    raise ValueError(f"Invalid uniform distribution for {key}. 'loc' or 'scale' is missing.")
                params[key.split("__")[1]] = trial.suggest_float(key, float(loc), float(loc + scale))
            else:
                raise ValueError(f"Unsupported scipy.stats distribution: {type(value.dist)}")
        elif isinstance(value, tuple) and len(value) == 2:  # Handle (low, high) tuples
            params[key.split("__")[1]] = trial.suggest_float(key, value[0], value[1])
        else:
            raise ValueError(f"Unsupported parameter type: {type(value)}")

    # Create the pipeline with the trial's parameters
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model.set_params(**params))])

    # Perform stratified cross-validation
    scores = []
    for train_idx, val_idx in stratified_cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]


        pipeline.fit(X_train_fold, y_train_fold)
        y_val_scores = pipeline.predict_proba(X_val_fold)[:, 1]
        score = recall_at_fpr(y_val_fold, y_val_scores, target_fpr=0.05)
        scores.append(score)
    
    return np.mean(scores)

# Optimize selected models
for name in models:
    model_config = models[name]
    model = model_config["model"]
    param_distributions = model_config["param_distributions"]
    print(f"\n--- Starting Optuna optimization for {name} ---\n")

    # Instantiate the model based on the model name
    if name == "LightGBM":
        model = LGBMClassifier()
    elif name == "XGBoost":
        model = XGBClassifier()
    else:
        raise ValueError(f"Unsupported model: {name}")

    # Start timing
    start_time = time.time()

    # Define the study name and filename
    study_name = f"study_{name}"
    filename = f"saved_models/{study_name}.joblib"

    # Load existing study or create a new one
    if os.path.exists(filename):
        study = joblib.load(filename)
        print(f"Loaded existing study for {name}.")
    else:
        study = optuna.create_study(direction="maximize", study_name=study_name)
        print(f"Created new study for {name}.")

    # Define a trial callback to save progress after each trial
    def trial_callback(study, trial):
        save_study_progress(study, filename)
        print(f"Study for {name} saved after trial {trial.number}.")

    # Optimize the study
    study.optimize(
        lambda trial: objective(trial, name, model, param_distributions, X_train, y_train),
        n_trials=50,
        callbacks=[trial_callback]
    )
    
    # Get the best parameters and model
    best_params = study.best_params
    best_score = study.best_value
    print(f"Best parameters for {name}: {best_params}")
    print(f"Best score for {name}: {best_score}")

    # Save the final study
    save_study_progress(study, filename)
    print(f"Final study for {name} saved as {filename}.\n")

    elapsed_time = time.time() - start_time
    elapsed_time_str = f"{elapsed_time:.2f}s"
    
    best_models[name] = (model.set_params(**best_params), best_score)

# Optimize LightGBM using Optuna
study_lgbm = optuna.create_study(direction='maximize', storage=storage, study_name='lgbm_study', load_if_exists=True, sampler=sampler, pruner=pruner)
print("\n--- Starting Optuna optimization for LightGBM ---\n")
total_trials = 200
start_time = time.time()
for trial_idx in range(total_trials):
    trial_start_time = time.time()
    study_lgbm.optimize(objective_lgbm, n_trials=1)
    # Save intermediate model after each trial
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    intermediate_lgbm_filename = f"saved_models/LightGBM_trial_{trial_idx + 1}_{current_datetime}.joblib"
    print(f"Saving intermediate LightGBM model for trial {trial_idx + 1} as {intermediate_lgbm_filename}...\n")
    joblib.dump(pipeline_lgbm, intermediate_lgbm_filename)
    print(f"Intermediate LightGBM model for trial {trial_idx + 1} saved successfully as {intermediate_lgbm_filename}.\n")
    trial_end_time = time.time()
    trial_duration = trial_end_time - trial_start_time
    elapsed_time = trial_end_time - start_time
    remaining_trials = total_trials - (trial_idx + 1)
    eta_trials = remaining_trials * trial_duration
    print(f"\n--- Finished trial {trial_idx + 1}/{total_trials} for LightGBM in {trial_duration:.2f} seconds ---\n")
    print(f"Elapsed time: {elapsed_time:.2f} seconds. Estimated time remaining for all trials: {eta_trials:.2f} seconds\n")
end_time = time.time()
print(f"\n--- Finished Optuna optimization for LightGBM in {end_time - start_time:.2f} seconds ---\n")

# Save the best LightGBM model
lgbm_best_params = study_lgbm.best_params
pipeline_lgbm.set_params(**lgbm_best_params)
pipeline_lgbm.fit(X_train, y_train)
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
lgbm_filename = f"saved_models/LightGBM_optuna_{current_datetime}.joblib"
print(f"Saving best Optuna LightGBM model as {lgbm_filename}...\n")
joblib.dump(pipeline_lgbm, lgbm_filename)
print(f"Best Optuna LightGBM model saved successfully as {lgbm_filename}.\n")


[LightGBM] [Info] Number of positive: 7059, number of negative: 632941
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025063 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2318
[LightGBM] [Info] Number of data points in the train set: 640000, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011030 -> initscore=-4.496074
[LightGBM] [Info] Start training from score -4.496074
[LightGBM] [Info] Number of positive: 7059, number of negative: 632941
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2318
[LightGBM] [Info] Number of data points in the train set: 640000, number of used features: 46
[LightGBM] [Info

[I 2024-12-09 04:37:14,016] A new study created in memory with name: study_XGBoost



--- Starting Optuna optimization for XGBoost ---

Created new study for XGBoost.


[I 2024-12-09 04:37:53,922] Trial 0 finished with value: 0.4790884096792635 and parameters: {'classifier__max_depth': 9, 'classifier__learning_rate': 0.09000188442707203, 'classifier__n_estimators': 224, 'classifier__subsample': 0.8115890408643198, 'classifier__colsample_bytree': 0.8794835538963024, 'classifier__min_child_weight': 50, 'classifier__scale_pos_weight': 500, 'classifier__gamma': 4.713190292505617, 'classifier__reg_alpha': 0.8672643810586234, 'classifier__reg_lambda': 0.2975000969830429, 'classifier__objective': 'binary:logistic', 'classifier__eval_metric': 'aucpr'}. Best is trial 0 with value: 0.4790884096792635.


Study for XGBoost saved after trial 0.


...

Study for XGBoost saved after trial 47.


[I 2024-12-09 05:21:20,912] Trial 48 finished with value: 0.5321314550371612 and parameters: {'classifier__max_depth': 4, 'classifier__learning_rate': 0.0697647843778962, 'classifier__n_estimators': 600, 'classifier__subsample': 0.7949377224049576, 'classifier__colsample_bytree': 0.6791619558304807, 'classifier__min_child_weight': 10, 'classifier__scale_pos_weight': 50, 'classifier__gamma': 0.8660707473403996, 'classifier__reg_alpha': 0.47891542428776096, 'classifier__reg_lambda': 0.18166482069907292, 'classifier__objective': 'binary:logistic', 'classifier__eval_metric': 'aucpr'}. Best is trial 32 with value: 0.5380248341073918.


Study for XGBoost saved after trial 48.


[I 2024-12-09 05:22:11,400] Trial 49 finished with value: 0.5328106351133466 and parameters: {'classifier__max_depth': 3, 'classifier__learning_rate': 0.04496097576863797, 'classifier__n_estimators': 853, 'classifier__subsample': 0.8483380558727407, 'classifier__colsample_bytree': 0.8602913292657859, 'classifier__min_child_weight': 10, 'classifier__scale_pos_weight': 500, 'classifier__gamma': 0.2246049274320443, 'classifier__reg_alpha': 0.3689568444588318, 'classifier__reg_lambda': 0.0031195170257367066, 'classifier__objective': 'binary:logistic', 'classifier__eval_metric': 'aucpr'}. Best is trial 32 with value: 0.5380248341073918.


Study for XGBoost saved after trial 49.
Best parameters for XGBoost: {'classifier__max_depth': 3, 'classifier__learning_rate': 0.05234450595039155, 'classifier__n_estimators': 899, 'classifier__subsample': 0.5410434450411552, 'classifier__colsample_bytree': 0.6966875820615125, 'classifier__min_child_weight': 1, 'classifier__scale_pos_weight': 50, 'classifier__gamma': 0.35671465038041017, 'classifier__reg_alpha': 0.36971142945786795, 'classifier__reg_lambda': 0.0850666877231463, 'classifier__objective': 'binary:logistic', 'classifier__eval_metric': 'aucpr'}
Best score for XGBoost: 0.5380248341073918
Final study for XGBoost saved as saved_models/study_XGBoost.joblib.


--- All Models Processed ---



In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import joblib
from xgboost import XGBClassifier

def recall_at_fpr(y_true, y_scores, target_fpr):
    from sklearn.metrics import roc_curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    idx = np.where(fpr <= target_fpr)[0][-1] if np.any(fpr <= target_fpr) else None
    return tpr[idx] if idx is not None else 0.0

# Load the Optuna study and get the best parameters
study = joblib.load("./saved_models/study_XGBoost.joblib")
best_params = study.best_params
print("Best Parameters:", best_params)

# Create the best model using the best parameters
best_model = XGBClassifier(**best_params)

# Train the model
best_model.fit(X_test_preprocessed, y_test)

# Save the model
joblib.dump(best_model, "best_xgb_model.joblib")

# Load the saved model
loaded_model = joblib.load("best_xgb_model.joblib")

# Predict probabilities for evaluation
probabilities = loaded_model.predict_proba(X_test_preprocessed)[:, 1]

# Calculate AUC-ROC
auc_score = roc_auc_score(y_test, probabilities)

# Evaluate recall at a target FPR
target_fpr = 0.01  # Specify the desired FPR
recall = recall_at_fpr(y_test, probabilities, target_fpr)

print(f"AUC-ROC Score: {auc_score:.4f}")
print(f"Recall at FPR {target_fpr * 100:.1f}%: {recall:.2f}")


Best Parameters: {'classifier__max_depth': 3, 'classifier__learning_rate': 0.05234450595039155, 'classifier__n_estimators': 899, 'classifier__subsample': 0.5410434450411552, 'classifier__colsample_bytree': 0.6966875820615125, 'classifier__min_child_weight': 1, 'classifier__scale_pos_weight': 50, 'classifier__gamma': 0.35671465038041017, 'classifier__reg_alpha': 0.36971142945786795, 'classifier__reg_lambda': 0.0850666877231463, 'classifier__objective': 'binary:logistic', 'classifier__eval_metric': 'aucpr'}


/home/gamacore/FINAL/Bank-Fraud-Detection/bank/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [19:04:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "classifier__colsample_bytree", "classifier__eval_metric", "classifier__gamma", "classifier__learning_rate", "classifier__max_depth", "classifier__min_child_weight", "classifier__n_estimators", "classifier__objective", "classifier__reg_alpha", "classifier__reg_lambda", "classifier__scale_pos_weight", "classifier__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


AUC-ROC Score: 0.9878
Recall at FPR 1.0%: 0.78


## 3. Test data evaluation

We now review the performance of the best model on the test data. We will use the following metrics to evaluate the model: auc roc score, tpr @ fpr = 0.05, and predictive equality.

In [4]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import joblib

def decode_protected_attributes(X_original, X_preprocessed, preprocessor, attributes):
    """
    Decode the protected attributes from preprocessed data back to their original values.

    Parameters:
        X_original: The original unprocessed DataFrame.
        X_preprocessed: The preprocessed array.
        preprocessor: The ColumnTransformer used for preprocessing.
        attributes: List of attribute names to decode.

    Returns:
        dict: Decoded attributes with original values for fairness evaluation.
    """
    # Initialize dictionary to hold decoded attributes
    decoded_attributes = {}

    for attr in attributes:
        if attr in preprocessor.transformers_[0][1].onehot_features:
            # Decode one-hot-encoded attributes
            encoder = preprocessor.transformers_[0][1].onehot_encoder
            encoded_columns = encoder.get_feature_names_out([attr])
            indices = [
                i for i, col in enumerate(preprocessor.transformers_[0][1].get_feature_names_out())
                if col in encoded_columns
            ]
            decoded_values = np.argmax(X_preprocessed[:, indices], axis=1)
            decoded_attributes[attr] = encoder.categories_[0][decoded_values]
        elif attr in preprocessor.transformers_[0][1].ordinal_features:
            # Decode ordinal-encoded attributes
            encoder = preprocessor.transformers_[0][1].ordinal_encoder
            index = list(preprocessor.transformers_[0][1].ordinal_features).index(attr)
            decoded_values = X_preprocessed[:, index].astype(int)
            decoded_attributes[attr] = encoder.categories_[0][decoded_values]
        else:
            raise ValueError(f"Attribute {attr} is not recognized as encoded by the preprocessor.")
    return decoded_attributes

def evaluate_model(model, X_test, y_test, protected_attributes):
    """
    Evaluate a model using AUC ROC, TPR @ FPR = 0.05, and Predictive Equality across multiple attributes.
    """
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # AUC ROC Score
    auc_roc = roc_auc_score(y_test, y_pred_proba)
    
    # TPR @ FPR = 0.05
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    target_fpr = 0.05
    tpr_at_fpr = tpr[np.where(fpr <= target_fpr)[0][-1]] if np.any(fpr <= target_fpr) else 0.0
    
    # Predictive Equality
    fairness_results = {}
    for attr_name, attr_values in protected_attributes.items():
        groups = np.unique(attr_values)
        fpr_groups = {}
        for group in groups:
            group_idx = attr_values == group
            group_fpr, _, _ = roc_curve(y_test[group_idx], y_pred_proba[group_idx])
            fpr_groups[group] = group_fpr[1] if len(group_fpr) > 1 else 0.0
        fairness_results[attr_name] = fpr_groups

    return {"AUC ROC": auc_roc, "TPR @ FPR = 0.05": tpr_at_fpr, "Predictive Equality": fairness_results}

# Decode protected attributes
attributes_to_decode = ['customer_age', 'employment_status', 'income']
protected_attributes = decode_protected_attributes(X_test, X_test_preprocessed, preprocessor, attributes_to_decode)

# Load Optuna studies
lightgbm_study_path = './saved_models/study_LightGBM.joblib'
xgboost_study_path = './saved_models/study_XGBoost.joblib'

# Load and reconstruct LightGBM
try:
    lightgbm_study = joblib.load(lightgbm_study_path)
    lightgbm_best_params = lightgbm_study.best_params
    lightgbm_model = lgb.LGBMClassifier(**lightgbm_best_params)
    lightgbm_model.fit(X_train_preprocessed, y_train)
    print("LightGBM model reconstructed and trained.")
except Exception as e:
    print(f"Error with LightGBM model: {e}")
    lightgbm_model = None

# Load and reconstruct XGBoost
try:
    xgboost_study = joblib.load(xgboost_study_path)
    xgboost_best_params = xgboost_study.best_params
    xgboost_model = xgb.XGBClassifier(**xgboost_best_params, use_label_encoder=False)
    xgboost_model.fit(X_train_preprocessed, y_train)
    print("XGBoost model reconstructed and trained.")
except Exception as e:
    print(f"Error with XGBoost model: {e}")
    xgboost_model = None

# Evaluate models
if lightgbm_model:
    print("\n--- Evaluating LightGBM Model ---")
    lightgbm_results = evaluate_model(lightgbm_model, X_test_preprocessed, y_test, protected_attributes)
    print(lightgbm_results)

if xgboost_model:
    print("\n--- Evaluating XGBoost Model ---")
    xgboost_results = evaluate_model(xgboost_model, X_test_preprocessed, y_test, protected_attributes)
    print(xgboost_results)


AttributeError: 'Pipeline' object has no attribute 'transformers_'